# Imports
## Note: If making major changes, try on a copy of this notebook

In [1]:
!pip3 install --upgrade pip

     |████████████████████████████████| 2.1 MB 12.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [2]:
!pip3 install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.8/880.8 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 KB 11.5 MB/s eta 0:00:00


In [3]:
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pennylane as qml

# Pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


In [4]:
!pip3 install qiskit
!pip3 install pennylane-qiskit

from qiskit import Aer
import pennylane_qiskit

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.8/943.8 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Dataset Loader
### Completed (working on V3)     
@TODO: Gain ideas from MolGAN ✅  
@FIXED: `process` function not returning anything  
@UPDATE: Now loads all 11847 molecules into a giant array -> each array item contains details about one molecule  
@FIXED: `__getitem__` method not working as intended  
@UPDATE: each array item has the same size for generator (padded with dummy values)

In [5]:
class MoleculesLoader(torch.utils.data.Dataset):
    def __init__(self, csv_file: str, transform=None) -> None:
        self.csv = csv_file
        self.atom_dict = {"H" : 1, "C" : 2, "O" : 3, "N" : 4}
        self.transform = transform
        self.df = self.process()
        self._normalize()

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx) -> list:
        if torch.is_tensor(idx): idx = idx.tolist()
        return self.df[idx]

    def process(self) -> None:
        tmp_df = self.csv #not actually csv file but array
        df = [None] * len(tmp_df)
        for i in range(len(tmp_df)):
            mol_df = pd.read_csv(tmp_df[i])
            #print(f"mol_df={mol_df}")
            tmp_vec = [None] * len(mol_df)
            for j in range(len(mol_df)):
                data = mol_df.iloc[j]
                atom, x, y, z = data["atom"], data["x"], data["y"], data["z"]
                #print(f"Atom={self.atom_dict[atom]}, x={x}, y={y}, z={z}.")
                tmp_vec[j] = (self.atom_dict[atom], x, y, z)
            df[i] = tmp_vec
        return df

    def _normalize(self):
        maxlen = max([len(x) for x in self.df])
        self.size = maxlen
        for i in range(len(self.df)):
            while len(self.df[i]) != maxlen:
                self.df[i].append([0, 0.0, 0.0, 0.0])

In [6]:
csv = [f"/content/mol_xyz_{i}.csv" for i in range(int(input()))] 

31


In [7]:
Loaded = MoleculesLoader(csv)

In [8]:
Loaded.size

14

# Generator
## Optimized for generating molecules (random decimals)
@Task Deadline: Wednesday  
@UPDATE: Given current Generator problems probably going to take a *long* time.    
@UPDATE: Fixed on 03/15/22    
@NOTE: Do not use 32+ qubits (too much RAM)

In [9]:
n_qubits = 20  # Total number of qubits / N
n_a_qubits = 1  # Number of ancillary qubits / N_A
q_depth = 1  # Depth of the parameterised quantum circuit / D #try 1 for now

#We dont need this line anymore
#n_generators = 1  # Number of subgenerators for the patch method / N_G well obviously 1

In [10]:
#dev = qml.device("lightning.qubit", wires=n_qubits)
#dev = qml.device("qiskit.aer", wires=n_qubits)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #NTS: use GPU runtime

dev = qml.device('qiskit.aer', wires=n_qubits, shots=1024, backend='qasm_simulator')
dev

<AerDevice device (wires=20, shots=1024) at 0x7f4702101e50>

In [11]:
device

device(type='cpu')

Uncomment the comments to add more complexity and/or parametrized gates

In [12]:
@qml.qnode(dev, interface="torch", diff_method="parameter-shift")
def quantum_circuit(noise, weights):
    weights = weights.reshape(q_depth, n_qubits)
    #Optional: superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.CNOT(wires=[i, (i+1) % n_qubits])

    # Initialise latent vectors using noise
    for i in range(n_qubits):
        qml.RY(noise[i], wires=i)

    # Repeated layer
    for i in range(q_depth):
        # Parameterised layer
        for y in range(n_qubits):
            qml.RY(weights[i][y], wires=y)
            #Optional: more parameters
            #qml.RX(weights[i][y], wires=y)
            #qml.RZ(weights[i][y], wires=y)

        # Control Z gates
        for y in range(n_qubits - 1):
            qml.CZ(wires=[y, y + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [13]:
#returns a list of 1 and 0 based on probs
def partial_measure(noise, weights):
    probs = quantum_circuit(noise, weights)
    bits = ["0" if x < 0 else "1" for x in probs]
    return bits

In [14]:
tmp_noise = torch.rand(n_qubits, device=device) * math.pi / 2

In [15]:
sussy = nn.ParameterList([nn.Parameter(torch.rand(q_depth * n_qubits), requires_grad=True)])

In [16]:
sussy[0]

Parameter containing:
tensor([0.9299, 0.9443, 0.2301, 0.8877, 0.4120, 0.7036, 0.1439, 0.2814, 0.6416,
        0.3434, 0.7826, 0.0691, 0.9335, 0.0469, 0.2788, 0.5700, 0.9092, 0.8311,
        0.4761, 0.2211], requires_grad=True)

In [17]:
tmp_noise

tensor([0.5456, 0.9492, 0.0710, 1.0976, 1.3463, 1.1506, 0.5016, 1.5605, 0.2225,
        0.1370, 1.3486, 1.4458, 0.1459, 0.2121, 0.1912, 0.3289, 1.0178, 0.8608,
        0.7249, 0.7891])

In [18]:
bruh = partial_measure(tmp_noise, sussy[0])

In [19]:
''.join(bruh)

'10110101101101010101'

@TODO: ``forward()` function needs to be cmpletely modified to work for random noise input.  
@UPDATE: Given this, it'll probably take at least 2 days (*expect Wednesday*) to complete but even then functionability not guaranteed    
@UPDATE: Fixed on Tuesday

In [20]:
class QuantumGenerator(nn.Module):
    """Quantum generator class"""

    def __init__(self, q_delta=1):
        """
        Args:
            n_generators (int): Number of sub-generators to be used in the patch method.
            q_delta (float, optional): Spread of the random distribution for parameter initialisation.
        """

        super().__init__()

        self.q_params = nn.ParameterList(
            [
                nn.Parameter(q_delta * torch.rand(q_depth * n_qubits), requires_grad=True)
            ]
        )

    def forward(self, x):
        molecules = []
        for params in self.q_params:
            #for elem in x:
            q_out = partial_measure(x, params) #q_out is a list of 1 and 0
            bitstr = "".join(q_out)
            molecules.append(bitstr)
        return molecules

In [21]:
lrG = 0.3  # Learning rate for the generator
lrD = 0.01  # Learning rate for the discriminator
num_iter = 1  # Number of training iterations

In [22]:
tmp_gen = QuantumGenerator().to(device)

In [23]:
tmp_out = tmp_gen.forward(tmp_noise)

In [24]:
tmp_out

['01111010100110001100']

# Post Generator / Pre Discriminator Processing
### Completed
@Task Deadline: Monday ✅  
@UPDATE: Wrapped in a class for better UX

In [31]:
class Processing(object):
    def __init__(self, noab: int=2, nocb: int=6) -> None:
        self.pi = np.pi
        self.atom_dict = {"00" : 1, "01" : 2, "10" : 3, "11" : 4}
        self.num_of_atoms = 1
        self.bits_per_atom = noab
        self.bits_per_coord = nocb
        self.sign_bit = 1
        self.num_of_total_bits = self.num_of_atoms * (
            self.bits_per_atom + 3 * (self.sign_bit + self.bits_per_coord)
        )


    def whichAtom(self, atom: str) -> str:
        try: x = self.atom_dict[atom]
        except KeyError: raise Exception(f"Key: {atom} is not in atom_dict!")
        else: return x 

    def calcDistance(self, coord_dist, num_of_qubits: int, sign: bool) -> float:
        distance = float(int(coord_dist, 2) / pow(2, num_of_qubits-2))
        return (distance * -1 if sign else distance)

    def atomsAndCoordinates(self, generatedVector: str) -> list:
        genVec = generatedVector
        atomBS = genVec[0:self.bits_per_atom]
        signx = (True if genVec[0] == "1" else False)
        xcoord = genVec[self.bits_per_atom:self.bits_per_atom+self.bits_per_coord]
        signy = (True if genVec[self.bits_per_atom+self.bits_per_coord] == "1" else False)
        ycoord = genVec[1+self.bits_per_atom+self.bits_per_coord:1+self.bits_per_atom+self.bits_per_coord*2]
        signz = (True if genVec[1+self.bits_per_atom+self.bits_per_coord*2] == "1" else False)
        zcoord = genVec[2+self.bits_per_atom+self.bits_per_coord*2:]
        atom = self.whichAtom(atomBS)
        xdist = self.calcDistance(xcoord, self.bits_per_coord, signx)
        ydist = self.calcDistance(ycoord, self.bits_per_coord, signy)
        zdist = self.calcDistance(zcoord, self.bits_per_coord, signz)
        return [atom, xdist, ydist, zdist]


In [32]:
PRO = Processing(nocb=5)

In [33]:
tmp_pro = [PRO.atomsAndCoordinates(out) for out in tmp_out]

In [34]:
tmp_pro = torch.tensor(tmp_pro)

In [35]:
tmp_pro

tensor([[2.0000, 3.6250, 2.3750, 1.5000]])

In [36]:
PRO.num_of_total_bits == n_qubits

True

# Discriminator
### Optimized towards molecules instead of images
@Task Deadline: **COMPLETED**  
@TODO: Check how MolGAN people loaded the molecules ✅  
@UPDATE: Added options to save and load model  
@UPDATE: Changed model architecture  
@FIXED: `TypeError: linear(): argument 'input' (position 1) must be Tensor, not list`   
@PROBLEM: `torch.tensor()` doesn't work on lists with more than one `dtype`  
@FIXED: ⬆️ use ASCII value of string `(dtype=int)`.  
@UPDATE: Fixed issues with Doubles and Floats   
@UPDATE: Add capabilities for dynamic-sized inputs

In [37]:
class Discriminator(nn.Module):
    def __init__(self, data_shape):
        super(Discriminator, self).__init__()
        self.data_shape = data_shape

        self.model = nn.Sequential(
            #MORE RAM
            #nn.Linear(int(np.prod(self.data_shape)), 262144),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(262144, 512),
            #nn.LeakyReLU(0.2, inplace=True),
            #nn.Linear(512, 1),

            nn.Linear(4, int(np.prod(self.data_shape))),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(56, 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )

    def forward(self, mol):
        #print(len(mol), len(mol[0]))
        validity = self.model(mol.float())
        return validity

    def save(self, path):
        save_dict = {
            'model': self.model.state_dict(),
            'data_shape': self.data_shape,
        }
        torch.save(save_dict, path)
        return

    @staticmethod
    def load(path):
        save_dict = torch.load(path)
        D = Discriminator(save_dict['data_shape'])
        D.model.load_state_dict(save_dict["model"])

        return D

In [38]:
D = Discriminator(data_shape=(Loaded.size, 4))

In [39]:
D.model

Sequential(
  (0): Linear(in_features=4, out_features=56, bias=True)
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Linear(in_features=56, out_features=8, bias=True)
  (3): LeakyReLU(negative_slope=0.2, inplace=True)
  (4): Linear(in_features=8, out_features=1, bias=True)
  (5): Sigmoid()
)

In [40]:
ttsx = D.forward(torch.tensor(Loaded.df[0], dtype=torch.double, requires_grad=True)).view(-1)

In [60]:
ttsx

tensor([0.4521, 0.4471, 0.4672, 0.4661, 0.4579, 0.4550, 0.4550, 0.4550, 0.4550,
        0.4550, 0.4550, 0.4550, 0.4550, 0.4550], grad_fn=<ViewBackward0>)

# Loss Function

In [41]:
class MoleculeLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(MoleculeLoss, self).__init__()
        self.actual_loss_func = nn.BCELoss()

    def forward(self, inputs, targets, smooth=1, device=None):
        if not inputs.requires_grad: inputs.requires_grad = True
        if not targets.requires_grad: targets.requires_grad = True
        #supposedly Wasserstein loss
        #inputs = #F. [function]
        inputs = inputs.view(-1)
        inputs = torch.tensor(list(map(lambda x: x - int(x) if x >= 1 else -x + int(x) if x < 0 else x, inputs)))
        #print(inputs)
        targets = targets.view(-1)
        #targets = list(map(lambda x: x - int(x) if x >= 0 else -x - int(x)), targets) #no need
        return self.actual_loss_func(inputs.to(device), targets)

In [42]:
real = torch.full((Loaded.size,), 1.0, dtype=torch.float, device=device)

In [43]:
real

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [44]:
ML = MoleculeLoss()
#ML = nn.BCELoss()

In [45]:
ttsx

tensor([0.4521, 0.4471, 0.4672, 0.4661, 0.4579, 0.4550, 0.4550, 0.4550, 0.4550,
        0.4550, 0.4550, 0.4550, 0.4550, 0.4550], grad_fn=<ViewBackward0>)

In [46]:
ratiod = ML(ttsx, real, device=device)

In [47]:
print(f"Loss: {ratiod:.9f}")

Loss: 0.785129488


# Putting it all together / training
### Very Incomplete
@Task Deadline: Wednesday    
@Update: Fix Training Loop   
@TODO: Specify `batch_size`  
@TODO: Include postprocessing in loop

In [61]:
discriminator = Discriminator(data_shape=(Loaded.size, 4)).to(device)
generator = QuantumGenerator().to(device)
processor = Processing(nocb=5)

In [62]:
criterion = MoleculeLoss()

# Optimisers
optD = optim.SGD(discriminator.parameters(), lr=lrD)
optG = optim.SGD(generator.parameters(), lr=lrG)

batch_size = Loaded.size

real_labels = torch.full((batch_size,), 1.0, dtype=torch.float, device=device)
fake_labels = torch.full((batch_size,), 0.0, dtype=torch.float, device=device)

# Fixed noise allows us to visually track the generated MOLECULES throughout training
fixed_noise = torch.rand(n_qubits, device=device) * math.pi / 2

# Iteration counter
counter = 0

In [50]:
criterion

MoleculeLoss(
  (actual_loss_func): BCELoss()
)

In [51]:
real_labels

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [52]:
fake_labels

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [79]:
ratom_dict = {1: "H", 2: "C", 3 : "O", 4: "N"}
def look_good(data) -> None:
    #print(data)
    for d in data:
        if int(d[0]) == 0: continue
        print(f"Atom={ratom_dict[int(d[0])]},\tx={d[1]:.9f},\ty={d[2]:.9f},\tz={d[3]:.9f}")

In [77]:
num_iter = 31
counter = 0

In [75]:
moleculelength = int(input(f"Enter a number from 1 to {Loaded.size}: "))

Enter a number from 1 to 14: 5


In [80]:
for i in range(len(Loaded)):
    data = Loaded.df[i]
    data = torch.tensor(data, dtype=torch.double, requires_grad=True)
    # Data for training the discriminator
    real_data = data.to(device)

    # Noise follwing a uniform distribution in range [0,pi/2)
    #noise = torch.rand(n_qubits.size, device=device) * math.pi / 2
    fake_data = []
    for _ in range(moleculelength):
        tmp_data = generator(fixed_noise)[0]
        tmp_data = processor.atomsAndCoordinates(tmp_data)
        #tmp_data = torch.tensor(tmp_data, requires_grad=True).to(device)
        fake_data.append(tmp_data)
    for _ in range(Loaded.size - moleculelength):
        fake_data.append([0.0, 0.0, 0.0, 0.0])
    fake_data = torch.tensor(fake_data, requires_grad=True).to(device)

    #print(fake_data)
    #break
    # Training the discriminator
    discriminator.zero_grad()
    outD_real = discriminator(real_data)#.view(-1)
    outD_fake = discriminator(fake_data)#.view(-1)
    #outD_fake = [abs(breh) - abs(int(breh)) for breh in outD_fake]
    #outD_real = [abs(breh) - abs(int(breh)) for breh in outD_real]

    errD_real = criterion(torch.tensor(outD_real, requires_grad=True).to(device), real_labels)
    errD_fake = criterion(torch.tensor(outD_fake, requires_grad=True).to(device), fake_labels)

    #for j in range(5):
        #print(f"outD_real is {torch.tensor([outD_real[j]])}\n outD_fake is {torch.tensor([outD_fake[j]])}")
    #    errD_real += criterion(torch.tensor([abs(outD_real[j])]).to(device), real_labels)
    #    errD_fake += criterion(torch.tensor([abs(outD_fake[j])]).to(device), fake_labels)

    errD_real = torch.tensor(errD_real, requires_grad=True).to(device)
    errD_fake = torch.tensor(errD_fake, requires_grad=True).to(device)

    # Propagate gradients
    errD_real.backward()
    errD_fake.backward()

    errD = errD_real + errD_fake
    optD.step()

    # Training the generator
    generator.zero_grad()
    outD_fake = discriminator(fake_data).view(-1)
    #outD_fake = [abs(breh) - abs(int(breh)) for breh in outD_fake]
    errG = criterion(torch.tensor(outD_fake).to(device), real_labels)
    errG = torch.tensor(errG, requires_grad=True).to(device)
    errG.backward()
    optG.step()

    counter += 1

    # Show loss values
    if counter % 3 == 0:
        print(f'Iteration: {counter}, Discriminator Loss: {errD:0.6f}, Generator Loss: {errG:0.6f}')
        print(f"Generated Molecules:")
        look_good(fake_data)
        print()

    if counter == num_iter:
        break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tru

Iteration: 6, Discriminator Loss: 1.401731, Generator Loss: 0.659221
Generated Molecules:
Atom=C,	x=0.875000000,	y=0.875000000,	z=7.250000000
Atom=O,	x=-3.875000000,	y=3.125000000,	z=-0.500000000
Atom=N,	x=-1.250000000,	y=-3.875000000,	z=5.125000000
Atom=H,	x=0.500000000,	y=2.250000000,	z=5.375000000
Atom=N,	x=-3.125000000,	y=-2.750000000,	z=-0.125000000

Iteration: 9, Discriminator Loss: 1.401593, Generator Loss: 0.653388
Generated Molecules:
Atom=O,	x=-2.125000000,	y=0.875000000,	z=-6.000000000
Atom=C,	x=0.625000000,	y=0.625000000,	z=-0.875000000
Atom=O,	x=-3.625000000,	y=0.375000000,	z=4.750000000
Atom=O,	x=-1.625000000,	y=2.250000000,	z=-2.625000000
Atom=C,	x=2.000000000,	y=2.625000000,	z=-7.125000000

Iteration: 12, Discriminator Loss: 1.371544, Generator Loss: 0.675726
Generated Molecules:
Atom=O,	x=-0.750000000,	y=-2.375000000,	z=2.000000000
Atom=O,	x=-0.375000000,	y=0.375000000,	z=-2.250000000
Atom=C,	x=0.875000000,	y=-0.125000000,	z=2.000000000
Atom=O,	x=-2.125000000,	y=-3.375

In [ ]:
q = int(input().strip())

5


In [ ]:
print(f"We will need {q*processor.num_of_total_bits} qubits to make a length {q} molecule")

We will need 100 qubits to make a length 5 molecule
